In [18]:
from atproto import Client, client_utils
from dotenv import load_dotenv
import sqlite3
import time 
import os

load_dotenv()

username = os.getenv('username')
password = os.getenv('password')

client = Client()
client.login(username, password)

ProfileViewDetailed(did='did:plc:oc7nomnf7bsgmraom72ts3fp', handle='alex-paul-kelly.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:oc7nomnf7bsgmraom72ts3fp/bafkreiasgk56xkl76blphf7uo2yprnv45smrs56w4vouxphmaav6jibpzm@jpeg', banner=None, created_at='2024-11-19T13:11:41.438Z', description=None, display_name='', followers_count=15, follows_count=313, indexed_at='2024-11-19T13:11:41.438Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=5, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=KnownFollowers(count=8, followers=[ProfileViewBasic(did='did:plc:5hkogxu5u4wkelj6qm6iarip', handle='aisupremacy.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:5hkogxu5u4wkelj6qm6iarip/bafkreicianwa7fzkz5dcgnb344

In [7]:
# Set up SQLite database
conn = sqlite3.connect('bsky_posts.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        author TEXT,
        text TEXT
    )
''')
conn.commit()

print("Database setup complete. Starting feed...")

Database setup complete. Starting feed...


In [8]:
# Keep track of the most recent post we've seen
last_seen_post = None

while True:
    try:
        # Get the timeline
        timeline = client.get_timeline(limit=50)  # Adjust limit as needed

        # Iterate through posts, newest first
        for post in reversed(timeline.feed):
            # If we've seen this post before, skip to the next one
            if last_seen_post and post.post.cid == last_seen_post:
                break
            
            # Insert post into database
            cursor.execute('''
                INSERT INTO posts (author, text) VALUES (?, ?)
            ''', (post.post.author.handle, post.post.record.text))
            
            # # Print post details
            # print(f"Author: {post.post.author.handle}")
            # print(f"Text: {post.post.record.text}")
            # print("---")

        # Commit the changes to the database
        conn.commit()

        # Update the last seen post
        if timeline.feed:
            last_seen_post = timeline.feed[0].post.cid

        # Wait for a bit before checking again
        time.sleep(60)  # Wait for 60 seconds
        
    except KeyboardInterrupt:
        print("\nExiting the feed loop...")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Waiting before trying again...")
        time.sleep(300)  # Wait for 5 minutes before trying again

# Close the database connection
conn.close()
print("Database connection closed. Exiting program.")

# read the data 

In [29]:
# Connect to the database
conn = sqlite3.connect('bsky_posts.db')
cursor = conn.cursor()

# Read all posts from the database
cursor.execute('SELECT author, text FROM posts')
posts = cursor.fetchall()

In [31]:
# Display the posts
print(f"Total posts: {len(posts)}")
print("\nSample of posts:")
for post in posts[:5]:  # Show first 5 posts
    author, text = post
    print(f"Author: {author}")
    print(f"Text: {text}")
    print("-" * 50)

# Close the connection
conn.close()

Total posts: 3884

Sample of posts:
Author: avt.im
Text: A part of those dynamics is that more and more papers are getting submitted to ML venues, while other venues grow slower if at all.

Even people who repeatedly complain about ML venues still submit there, even work they could instead send to various journals. This isn’t by accident.
--------------------------------------------------
Author: neuripsconf.bsky.social
Text: NeurIPS Conference is now Live on Bluesky! 

-NeurIPS2024 Communication Chairs
--------------------------------------------------
Author: merve.bsky.social
Text: something small but might approaches
--------------------------------------------------
Author: nicoritschel.com
Text: Metrics for when updating multiple similar queries is annoying.

If code is more your thing, that's okay too & will be supported. Note the SQL— there's no new expression language to learn.
--------------------------------------------------
Author: nsaphra.bsky.social
Text: One of my favor

# pass the data to a llm 

In [32]:
#| echo: False
#| output: False

from dotenv import load_dotenv
import google.generativeai as genai
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
google_api_key = os.getenv('google_api_key')

In [33]:
genai.configure(api_key=google_api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [34]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)



In [61]:
my_bsky_posts = ""
for post in posts[:5]:
    my_bsky_posts += f"Author: {post[0]}\nText: {post[1]}\n"

In [62]:
print(my_bsky_posts) 


Author: avt.im
Text: A part of those dynamics is that more and more papers are getting submitted to ML venues, while other venues grow slower if at all.

Even people who repeatedly complain about ML venues still submit there, even work they could instead send to various journals. This isn’t by accident.
Author: neuripsconf.bsky.social
Text: NeurIPS Conference is now Live on Bluesky! 

-NeurIPS2024 Communication Chairs
Author: merve.bsky.social
Text: something small but might approaches
Author: nicoritschel.com
Text: Metrics for when updating multiple similar queries is annoying.

If code is more your thing, that's okay too & will be supported. Note the SQL— there's no new expression language to learn.
Author: nsaphra.bsky.social
Text: One of my favorite animations ever, but a bit distressing



In [63]:
llm_text = "this is a list of posts from blue sky, please analyze the data and return a list of topics that are being discussed/n" + my_bsky_posts

In [64]:
response = chat_session.send_message(llm_text)

print(response.text)


Analyzing the BlueSky posts reveals these topics:

1. **Machine Learning (ML) Conferences/Journals:**  The first post discusses the imbalance in submissions to ML venues versus other venues.  The NeurIPS post is a direct promotion of the conference.

2. **Software/Database:** The post from `nicoritschel.com` deals with metrics and SQL, indicating a focus on software development and potentially database management.

3. **Animation:** The post from `nsaphra.bsky.social` expresses a personal opinion about an animation.

4. **Social Media/Platform:** The NeurIPS post specifically mentions using Bluesky, which highlights the context of social media presence.


There isn't a single unifying theme across all posts, suggesting they were likely drawn from different discussions.

